import libraries


In [1]:
# Import necessary libraries
import pandas as pd
import os
import sys
import torch
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
import contractions

In [2]:
# import json

# # Read and print the content of the JSON file to inspect it
# file_path = "./A.json"
# with open(file_path, 'r') as f:
#     data = json.load(f)
#     data_list = list(data.items())
#     # print(data_list[:2])
#     # print(json.dumps(data, indent=4))  # Pretty-print the JSON content
#     print(type(data_list))
    
#     # Convert the JSON data to a list of dictionaries
#     list_of_dicts = [item[1] for item in data_list]
    
#     # Create a DataFrame from the list of dictionaries
#     df = pd.DataFrame(list_of_dicts)

#     print(df)

In [3]:
import os
import sys
import json
import pandas as pd
from sklearn.preprocessing import StandardScaler
import contractions

if not os.path.exists('./Atext') or not os.path.exists('./Models/Atext'):
    sys.exit()

# Define file paths for JSON data
file_path_1 = "./1.json"
file_path_2 = "./2.json"
file_path_3 = "./3.json"
file_path_4 = "./4.json"
file_path_5 = "./5.json"

# Initialize an empty DataFrame to store processed data
train_data_df = pd.DataFrame()

# Loop through each file to process training data
for file in [file_path_1, file_path_2, file_path_3, file_path_4]:
    # Check if the file exists
    if os.path.exists(file):
        # Read in training data from the JSON file
        with open(file, 'r') as f:
            data = json.load(f)
            data_list = list(data)
            # print(data_list[:2])
            list_of_dicts = [item for item in data_list]
            
            # Create a DataFrame from the list of dictionaries
            train_data = pd.DataFrame(list_of_dicts)
        
        print(f"Contents of {file}:")
        print(train_data.shape)
        
        # Preprocess and standardize labels
        train_data = train_data[train_data['PNEOA_scaled'] != '.']
        scaler = StandardScaler()
        train_data['PNEOA_scaled_new'] = scaler.fit_transform(train_data['PNEOA_scaled'].to_numpy().reshape(-1, 1))
        train_data = train_data[train_data['text'] != '']
        train_data['text'] = train_data['text'].apply(lambda x: contractions.fix(x))

        # Prepare DataFrame for training
        train_texts = train_data['text'].astype(str).tolist()
        train_labels = train_data['PNEOA_scaled_new'].astype(float).tolist()
        train_partids = train_data['PARTID'].astype(float).tolist()
        train_data_temp = pd.DataFrame({'PARTID': train_partids, 'text': train_texts, 'labels': train_labels})

        # Concatenate data to train_data_df
        train_data_df = pd.concat([train_data_df, train_data_temp], ignore_index=True)
        print(train_data_df.shape)
        print('\n')

    else:
        print("File does not exist:", file)

print(train_data_df.shape)

Contents of ./1.json:
(275, 52)
(269, 3)


Contents of ./2.json:
(275, 52)
(537, 3)


Contents of ./3.json:
(275, 52)
(809, 3)


Contents of ./4.json:
(275, 52)
(1076, 3)


(1076, 3)


In [4]:
# Initialize an empty DataFrame to store processed data
test_data_df = pd.DataFrame()
# Process test data from the fifth file
file = file_path_5
# Check if the file exists
if os.path.exists(file):
        # Read in training data from the JSON file
        with open(file, 'r') as f:
            data = json.load(f)
            data_list = list(data)
            # print(data_list[:2])
            list_of_dicts = [item for item in data_list]
            
            # Create a DataFrame from the list of dictionaries
            test_data = pd.DataFrame(list_of_dicts)
        
        print(f"Contents of {file}:")
        print(test_data.shape)
    
        # Preprocess and standardize labels
        test_data = test_data[test_data['PNEOA_scaled'] != '.']
        scaler = StandardScaler()
        test_data['PNEOA_scaled_new'] = scaler.fit_transform(test_data['PNEOA_scaled'].to_numpy().reshape(-1, 1))
        test_data = test_data[test_data['text'] != '']
        test_data['text'] = test_data['text'].apply(lambda x: contractions.fix(x))

        # Prepare DataFrame for training
        test_texts = test_data['text'].astype(str).tolist()
        test_labels = test_data['PNEOA_scaled_new'].astype(float).tolist()
        test_partids = test_data['PARTID'].astype(float).tolist()
        
        test_data_temp = pd.DataFrame({'PARTID': test_partids, 'text': test_texts, 'labels': test_labels})

        # Concatenate data to test_data_df
        test_data_df = pd.concat([test_data_df, test_data_temp], ignore_index=True)
        
else:
    print("File does not exist:", file)

print(test_data_df.shape)

Contents of ./5.json:
(275, 52)
(269, 3)


In [5]:
train_data_df["PARTID"]


0       1001.0
1       1002.0
2       1004.0
3       1007.0
4       1008.0
         ...  
1071    4228.0
1072    4229.0
1073    4230.0
1074    4231.0
1075    4232.0
Name: PARTID, Length: 1076, dtype: float64

In [6]:
int(0.95 * (train_data_df.shape[0]))

1022

In [7]:
len(train_data_df["PARTID"])

1076

In [8]:
train_data_df["PARTID"]

0       1001.0
1       1002.0
2       1004.0
3       1007.0
4       1008.0
         ...  
1071    4228.0
1072    4229.0
1073    4230.0
1074    4231.0
1075    4232.0
Name: PARTID, Length: 1076, dtype: float64

In [9]:
import random

# Setup logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Set up k-fold cross-validation
Results = []

# Initialize training and validation data
ids = random.sample(list(train_data_df["PARTID"]), int(0.95 * (train_data_df.shape[0])))
training_data = train_data_df[train_data_df["PARTID"].isin(ids)]
validation_data = train_data_df[~train_data_df["PARTID"].isin(ids)]

# Define model arguments
model_args = ClassificationArgs(
    sliding_window=True,
    use_early_stopping=True,
    early_stopping_metric="r2",
    early_stopping_metric_minimize=False,
    early_stopping_patience=5,
    num_train_epochs=50,
    learning_rate=2e-5,
    evaluate_during_training=True,
    regression=True,
    train_batch_size=16,
    eval_batch_size=8,
    evaluate_during_training_steps=1000,
    max_seq_length=512,
    no_cache=True,
    no_save=True,
    overwrite_output_dir=True,
    reprocess_input_data=True,
    gradient_accumulation_steps=2,
    save_best_model=True,
)

# Initialize and train the model
model = ClassificationModel(
    "roberta",
    "roberta-large",
    num_labels=1,
    args=model_args,
    use_cuda=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.train_model(training_data, eval_df=validation_data, r2=r2_score, mse=mean_squared_error, mae=mean_absolute_error)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_data_df, r2=r2_score, mse=mean_squared_error, mae=mean_absolute_error)
Results.append([result['r2'], result['mse'], result['mae']])


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1023 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4563 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1615 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1941 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 8274 features created from 1023 samples.


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Running Epoch 1 of 50:   0%|          | 0/518 [00:00<?, ?it/s]

...

  0%|          | 0/269 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (6326 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 269 features created from 269 samples.


Running Evaluation:   0%|          | 0/256 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'r2': 0.06831356684028667, 'mse': 0.9316864214452328, 'mae': 0.7656363899557393, 'eval_loss': 1.0338238241238287}


0.5863189697265625
-0.16692988077799478
-0.4484914143880208
-0.2511568069458008
0.4419759114583333
-0.10344386100769043
-0.033466339111328125
0.024755859375
0.043238321940104164
1.3450520833333333
0.909912109375
-0.034784490411931816
0.132403564453125
0.43509521484375
-0.54248046875
-0.570068359375
-0.0658721923828125
0.1895751953125
0.3063557942708333
0.4545574188232422
-0.136926952161287
-0.14790852864583334
1.0222981770833333
0.040995279947916664
0.499237060546875
-0.2794189453125
-0.73577880859375
0.02083411583533654
-0.05083920405461238
-0.15743001302083334
-0.44319915771484375
-0.0617218017578125
-0.14659423828125
-0.40062537560096156
0.24747212727864584
-0.04420267740885417
-0.017926897321428572
-0.65380859375
0.764892578125
0.22675791653719815
0.7596028645833334
-0.05599365234375
-0.18548583984375
0.5416782924107143
-0.476531982421875
-0.1285400390625
0.18578011648995535
0.00441741943359375
-0.4635467529296875
0.2860107421875
0.48294503348214285
-0.027492947048611112
-0.1405761

In [11]:
# Convert Results to a Pandas DataFrame for easier manipulation and saving
results_df = pd.DataFrame(Results, columns=['R2', 'MSE', 'MAE'])

# Calculate and print average results
avg_results = results_df.mean()
print(f"Average R2: {avg_results['R2']}, MSE: {avg_results['MSE']}, MAE: {avg_results['MAE']}")

# Define the directory where you want to save the results
save_directory = './Models/Atext/1/'

# Save to Excel
results_df.to_excel(save_directory + 'A_results.xlsx', index=False)

# Save to CSV
results_df.to_csv(save_directory + 'A_results.csv', index=False)

Average R2: 0.06831356684028667, MSE: 0.9316864214452328, MAE: 0.7656363899557393


In [12]:
# Save model, tokenizer, and configuration
model_save_path = "./Atext/1/"
model.save_model(model_save_path)
files = os.listdir(model_save_path)
print(files)
model.model.save_pretrained(model_save_path)
files = os.listdir(model_save_path)
print(files)
model.tokenizer.save_pretrained(model_save_path)
files = os.listdir(model_save_path)
print(files)
model.config.save_pretrained(model_save_path)
files = os.listdir(model_save_path)
print(files)
print("Model, tokenizer, and configuration saved successfully to", model_save_path)

[]
['model.safetensors', 'config.json']
['tokenizer.json', 'vocab.json', 'model.safetensors', 'config.json', 'merges.txt', 'special_tokens_map.json', 'tokenizer_config.json']
['tokenizer.json', 'vocab.json', 'model.safetensors', 'config.json', 'merges.txt', 'special_tokens_map.json', 'tokenizer_config.json']
Model, tokenizer, and configuration saved successfully to ./Atext/1/
